In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
class DNN:
    
    def __init__(self, layers):
        self.w = self._init_w(layers)
        self.b = self._init_b(layers)
    
    # initialize weights
    def _init_w(self, layers):
        w = []
        for i in range(len(layers)-1):
            w.append(np.random.randn(layers[i], layers[i+1]))
        return w
    
    # initialize biases
    def _init_b(self, layers):
        b = []
        for i in range(len(layers)-1):
            b.append(np.random.randn(1, layers[i+1]))
        return b
    
    # activation
    def _activate(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    # derivative of activation
    def _derivative(self, z):
        return self._activate(z) * (1-self._activate(z))
    
    # feed forward
    def _forward(self, x):
        x = x.astype('float32')
        z = [x.reshape(1, -1)]
        for i in range(len(self.w)):
            x = np.dot(x, self.w[i]) + self.b[i]
            z.append(x)
            x = self._activate(x)
        return x, z
    
    # backpropagation
    def _backprop(self, y, t, z):
        e = (y - t) * self._derivative(z[-1])
        delta_b = [e]
        delta_w = [self._activate(z[-2]).T @ e]
        for i in range(1, len(z)-1):
            e = np.dot(e, self.w[-i].T) * self._derivative(z[-(i+1)])
            delta_b.append(e)
            delta_w.append(self._activate(z[-(i+2)]).T @ e)
        return delta_w[::-1], delta_b[::-1]
    
    # evaluate the result
    def _evaluate(self, x, t):
        y, _ = self._forward(x)
        result = [(np.argmax(i), np.argmax(j)) for (i, j) in zip(y, t)]
        return sum(int(y == t) for (y, t) in result) / len(t)
    
    # train with batch
    def train(self, x, y, rate, epochs=100, batchsize=100):
        delta_w = [np.zeros(i.shape) for i in self.w]
        delta_b = [np.zeros(i.shape) for i in self.b]
        for i in range(epochs):
            idx = (np.random.sample(batchsize)*len(x)).astype(int)
            x_sample = x[idx]
            y_sample = y[idx]
            x_, z_ = self._forward(x_sample[0])
            delta_w, delta_b = self._backprop(x_, y_sample[0], z_)
            for j in range(1, len(y_sample)):
                x_, z_ = self._forward(x_sample[j])
                w_, b_ = self._backprop(x_, y_sample[j], z_)
                delta_w = [(ii + jj) for (ii, jj) in zip(delta_w, w_)]
                delta_b = [(ii + jj) for (ii, jj) in zip(delta_b, b_)]
            self.w = [(ii - rate*jj/len(x_sample)) for (ii, jj) in zip(self.w, delta_w)]
            self.b = [(ii - rate*jj/len(x_sample)) for (ii, jj) in zip(self.b, delta_b)]
            print (self._evaluate(x, y))

In [3]:
x = load_digits()['data']
y = load_digits()['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)
ytt = OneHotEncoder().fit_transform(y_train.reshape(-1,1)).toarray()

In [4]:
M = DNN([64, 16, 10])
M.train(x_train, ytt, 3, 1000, 10)

0.10390689941812137
0.0914380714879468
0.1288445552784705
0.15045719035743974
0.17206982543640897
0.19534497090606817
0.16458852867830423
0.15544472152950956
0.15544472152950956
0.14131338320864506
0.16043225270157938
0.16791354945968412
0.14962593516209477
0.16209476309226933
0.16458852867830423
0.16043225270157938
0.16375727348295926
0.200332502078138
0.22776392352452202
0.19118869492934332
0.19118869492934332
0.1862011637572735
0.21529509559434745
0.2227763923524522
0.2202826267664173
0.21862011637572734
0.22443890274314215
0.23857024106400665
0.2194513715710723
0.21612635078969245
0.23524522028262676
0.2344139650872818
0.2626766417290108
0.2709891936824605
0.27015793848711556
0.2601828761429759
0.2510390689941812
0.25519534497090607
0.23857024106400665
0.2660016625103907
0.2793017456359102
0.2826267664172901
0.23690773067331672
0.2518703241895262
0.257689110556941
0.25270157938487114
0.2543640897755611
0.25519534497090607
0.2593516209476309
0.256857855361596
0.257689110556941
0.239